In [1]:
import os
import json
import time
import requests
import random
import validators
import pickle
import json
import pickle

from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

In [2]:
# Read the search keywords and frontier URLs set
with open('../data/search_keywords.json', 'r') as fp:
    keywords = json.load(fp)

with open('../data/frontier_urls_initial.json', 'r') as fp:
    frontier_urls = json.load(fp)

print("Number of keywords: ", len(keywords))

Number of keywords:  4715


#### Creating frontier URLs list

We now expand the list of frontier URLs using the keywords

In [3]:
headers = {
    "Ocp-Apim-Subscription-Key": os.getenv("BING_API_KEY"),
    "X-Search-Location": "lat:48.31273;long:-9.03201;re:50"
}

def get_search_results(query):

    try:
        if "tübingen" not in query.lower():
            query += " Tübingen"

        query += ' language:en'
        
        params = {"q": query,  'responseFilter': 'Webpages', 'count': 30}
        response = requests.get(os.environ['BING_ENDPOINT'], headers=headers, params=params)
        search_results = response.json()

        # wait for 1 second to avoid rate limiting
        time.sleep(1)
        return search_results['webPages']['value']
    
    except Exception as e:
        print("Error: ", e)
        return []


idx = 888
print("Keyword: ", keywords[idx])
search_results = get_search_results(keywords[idx])   # still need english results!
search_results

Keyword:  Eurofins Institut Jäger Tübingen


[{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0',
  'name': 'Eurofins Institut Jäger GmbH Tübingen - Eurofins Deutschland',
  'url': 'https://www.eurofins.de/%C3%BCber-uns/standorte/eurofins-institut-jaeger-gmbh-tuebingen/',
  'isFamilyFriendly': True,
  'displayUrl': 'https://www.eurofins.de/über-uns/standorte/eurofins-institut-jaeger-gmbh-tuebingen',
  'snippet': 'Eurofins Institut Jäger GmbH Tübingen Ernst-Simon-Straße 2-4 D-72072 Tübingen GERMANY Phone: +49 7071 70070 Fax: +49 7071 700777 umwelt-tuebingen@eurofins.de www.eurofins-umwelt.de. Route berechnen: Von. Kontakt. Startseite | Suche ...',
  'dateLastCrawled': '2024-06-20T17:37:00.0000000Z',
  'language': 'en',
  'isNavigational': False,
  'noCache': True},
 {'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.1',
  'name': 'Eurofins Institut Jäger Tübingen - Eurofins Scientific',
  'url': 'https://www.eurofins.com/contact-us/worldwide-interactive-map/germany/eurofins-institut-jaeger-tuebingen/',
  'datePublishe

In [4]:
# Use multiprocessing to create 100 parallel processes to fetch search results
# map each keyword to get_search_results function

import multiprocess as mp

pool = mp.Pool(90)
search_results = pool.map(get_search_results, keywords)

Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'


In [5]:
print("Obtrained {} search results".format(len([x for x in search_results if x])))

Obtrained 4702 search results


In [6]:
keyword_results = zip(keywords, search_results)

# Save the search results to a pickle file
with open('../data/search_results.pkl', 'wb') as fp:
    pickle.dump(keyword_results, fp)

In [7]:
# combine the URLs from the search results and the frontier URLs

frontier_urls = set(frontier_urls)
for keyword, results in keyword_results:
    for result in results:
        url = result['url']
        frontier_urls.add(url)


print("Number of URLs in the frontier: ", len(frontier_urls))

Number of URLs in the frontier:  51713


In [8]:
# Write the updated frontier URLs to a pickle file

with open('../data/frontier_urls.pkl', 'wb') as fp:
    pickle.dump(frontier_urls, fp)

In [9]:
# count the number of URLs in result set
all_results = sum(search_results, [])

print("Total number of URLs in search results: ", len(all_results))
print("Number of unique URLs in search results: ", len(set([x['url'] for x in all_results])))

Total number of URLs in search results:  107566
Number of unique URLs in search results:  50311


In [ ]:
with open('../data/search_results.pkl', 'rb') as fp:
    search_results = pickle.load(fp)

for result in search_results:
    for snippet in result[1]:
        print(snippet['snippet'])
        break

Offizielle Webseite der Universitätsstadt Tübingen mit Informationen zu Politik, Kultur, Verwaltung, Bürgerservice, Klima und Umwelt, Tourismus
Das Schmiedtor war eines der fünf Stadttore der alten Stadtmauer Tübingens. Es stand am nördlichen Ende der Schmiedtorstraße in unmittelbarer Nähe zur Ammer in der Tübinger Unterstadt. Es wurde 1831 abgerissen. [1] Quellen [ Bearbeiten | Quelltext bearbeiten]
Einblicke in die Zahnarztpraxis in Tübingen. Gemeinschaftspraxis Dr. Christine Beyer & Jörg Potthoff befinden sich in Tübingen (Herrenbergerstr. 85). Sie ist dank der guten Lage und den zahlreichen Parkmöglichkeiten in der Nähe leicht zu erreichen. So ist für Sie und Ihre Familie eine unkomplizierte Anfahrt gewährleistet - für eine ...
Das Samariterstift liegt am Rand des Naturparks Schönbuch zwischen Tübingen und Herrenberg. Das Haus bietet eine hohe Aufenthaltsqualität: Ein großzügiger lichtdurchfluteter Speisesaal lädt die Bewohnerinnen und Bewohner zum Mittag- und Abendessen ein und di

In [19]:
count = 0
for keyword in keywords:
    if 'tübingen' in keyword.lower():
        count += 1

print("Number of keywords with Tübingen: ", count)

Number of keywords with Tübingen:  4348
